# Integrated Numerical Model for the Water Quality of the Forum Ponds

#### M.K. van der Molen, E. Peeters and R. Dijksma

## Contents

1. Introduction

2. Water quality      

3. Formulating the water quality model
   
4. Is it safe to swim in the campus ponds?

5. Finish the procedure

## 1 Introduction

In Integration Skills 2 - Water Quality, you will focus on extending the integrated numerical water balance model of the Forum ponds on the Wageningen University campus you built in *Integration Skills 1*. Again, you will describe the Forum ponds - or your group's land unit, but now focusing on water quality. 

In Integration Skills 1, you developed a quantitative, time explicit, water balance model. You composed the model formulation with simple terms. Later, you paid attention to better describing evaporation $E$, and lateral and vertical water flows $Q$-terms. You found that the water volume in the pond has a seasonal cycle, which changes depending on the terms in the water balance. You also spent time on the sensitivity of the model to the value of the initail volume and lateral inflow.

Note that the "Jupyter notebook" practical consists of so-called code cells and markdowns cells. Markdown cells are used to write text, pose questions, and for you to write your notes. Code cells are used to write parts of a numerical code or program. In each code cell we write a necessary part of a code and we write some comments that start with the mark "#". In the comment, we explain (again) what we calculate or plot. To obtain a result of a calculation i.e., a figure (note that most of the calculations we have already done for you), you will need to run a cell. You can do that by clicking on a cell and pressing simultaneously **Shift Enter**. Only then you will be able to see the result of the calculation and only then will the figures appear on your screen. Happy notebooking!

<p style="background:powderblue;color:black">(Python coding explanation:) > Some parts of the notebook are again marked in (powder)blue. These parts give some more in-depth explanation about coding in Python/Jupyter Notebooks.<br></p>

___
## Setting up Python modules

We start the tutorial by loading the required Python packages. In the cell below we load the necessary python packages (if some warning signs or requirement signs appear, ignore them).

In [ ]:
# Load necessary python packages.
#import sys
#!{sys.executable} -m pip install cufflinks > /dev/null; # Remove > /dev/null in case of errors.

import nemo
from ipywidgets import interact
import numpy as np
from numpy import log as ln
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf
import os

## 2 Water quality

The water quality may be described in a similar way as the water quantity, using a mass balance of the nutrients or pollutants in the water. It is important to realise that we are interested in the concentration, but that it is difficult to formulate a conservation equation for concentration. For example, if the pond’s water volume changes because of addition of clean rainwater, the concentration changes. But at the same time the total amount of nutrients in the pond stays the same. Therefore, we will formulate the model in terms of mass.

Today, you will describe the nitrogen cycle of the Forum ponds, including ammonium (NH$_3$) and nitrate (NO$_3$). We will not consider the Phosphorus cycle, that you learned how to quantify in the Basic Skills Soil Quality. Although the Phosphorus cycle would be very relevant, it would make the model too complex for now. 

After constructing the nitrogen balance model, you will apply it to verify the seasonal cycle of water quality, the risk for green or blue algae using the Redfield ratio, and the pond’s transparency in terms of Secchi depth. Ultimately, you will quantify how many people can healthily swim in the Forum ponds, considering the risk of bacterial diseases.

### 2.1 Nitrogen conservation equation

The terms in the nitrogen balance equation are partly comparable to the water quantity balance we learned about in Integration Skills 1:

\begin{equation*}
\frac{dm_n}{dt} = P \times C_{N,P}+Q_{k,in} \times C_{N,Q_{k,in}}-Q_{k,out} \times C_{N,pond}+Q_{l,in} \times C_{N,Q_{l,in}}-Q_{l,out} \times C_{N,pond}+F_{NH_3}+F_{den}+F_{exc}+\Delta m_{N,bio}.
\end{equation*}

The components of this equation are nitrogen in precipitation $P$ (mm s$^{-1}$) and nitrogen in lateral $Q_{l,in}$ (mm s$^{-1}$) and vertical inflow $Q_{k,in}$ (mm s$^{-1}$). The nitrogen concentrations $C_{N,P}$, $C_{N,Q_{k,in}}$, and $C_{N,Q_{l,in}}$ in those inputs are determined externally. You measured those as a part of Basic Skills Aquatic Ecology (see map in the introductory lecture on Brightspace). Nitrogen outputs from the pond are lateral and vertical outflows $Q_{l,out}$ and $Q_{k,out}$ (mm s$^{-1}$). The concentrations in these outputs depend on the concentration in the pond, which is the very variable you are trying to resolve. Nitrogen can also be added by dry deposition $F_{NH_3}$ (that you analysed in Basic Skills Air Quality), by denitrification $F_{den}$ (decomposition of organic matter) or by excreta from geese, ducks, gulls and other waterfowl ($F_{exc}$). Additionally, nitrogen may be fixed by aquatic vegetation and/or algae ($m_{N,bio}$). This nitrogen is removed from the water as ammonium or nitrate, but remains latently present until decomposition of the organic material. This term is somewhat complex, because algae may also be removed with lateral outflow.

Again, you need to integrate the partial differential equation to obtain the nitrogen mass in time:

\begin{equation*}
m_{N,t} = \int_{t_1}^{t_2}\frac{dm_N}{dt}dt.
\end{equation*}

Since integral is difficult to solve with computer models, you can solve it numerically as:

\begin{equation*}
m_{N,t+\Delta t} = m_{N,t}+\frac{dm_N}{dt}\times \Delta t.
\end{equation*}

You may see the similarity to the water quantity balance equation. There is one practical difference however: the water volume reacts rapidly to changes in inputs and outputs, and the maximum volume is fixed. The volume reaches equilibrium within one or two years. The nitrogen mass, however, reacts much more slowly. It may take five to ten years for it to reach equilibrium, or even longer. Moreover, the nitrogen mass does not have a maximum. Therefore, setting the initial concentration in the pond $C_{N,pond,ini}$ needs more attention than just setting the initial volume.

## 3 Formulating the water quality model

To construct the water quality model, we will follow the same procedure as for the water quantity model (used in the Integration Skills 1). Thus, first implement the tendency terms with simplistic values, then add up the tendency terms into $dm_N/dt$ and integrate the model in time. 

### 3.1 Tendency terms

Before we set up the initial condition for the model, in the cell box below, let's collect necessary parameters for the model. 

<ul>
<li><p style="background:powderblue;color:black">Parameters are settings to the model. They often represent choices you make. As such they are different from physical constants which are fixed in all conditions. In this situation, they are the time step `dt` [h], surface area of the pond A [m$^2$], maximum level of the pond `d_max` [m], deepest level of the pond `d_min` [m] and maximum volume of the pond `V_max` [m$^3$].
</li><br>

<li><p style="background:powderblue;color:black">Now, let's prepare some tendency terms. For that, you need the concentrations in the inflows (i.e., concentration in precipitation `C_N_P`, lateral inflow C_N_Ql_in and vertical inflow C_N_Qk_in).</ul>

**N.B.: Because of the C-situation, you weren't able to collect the measurements, but Edwin Peeters has been so kind to collect them for you. Please see the map on Brightspace --> Integration Course SWA --> Content --> Basic and Integration Skills --> Integration Skills 2 --> Integration Skills 2 Introduction.pdf, page 2.**

Note that you need to convert the concentration units from mg l$^{-1}$ to $\mu$g N m$^{-3}$.

In [ ]:
# Set up the parameters.
dt    = 1.                # h Set up the time-setp to 1 h.
A     = 5000.             # Set surface area of the pond (A; m^2).
d_max =  0.               # m  highest level of the pond (d_max; m) 
d_min = -3.               # m  deepest level of the pond (d_min; m) 
V_max = A*(d_max-d_min)   # m3 Calculate initial volume using A, d_max and d_min. 
V_ini = V_max             # m3 initial volume of the pond (assume full at the start)

# If the initial volume is bigger than the maximum volume of the pond, 
# set it to be equal to the maximum value.
if V_ini > V_max :
    V_ini = V_max
    
# Boundary conditions: Concentrations in inflowing water.
# You measured those in the Basic Skills Acquatic Ecology.
# Write them in order: concentration in precipitation C_N_P, concentration in lateral inflow C_N_l_in, 
# concentration in vertical inflow C_N_k_in. 
# All concentrations should be written in ug N/m3.
C_N_P     = 0.1e6   # ug/m3 Concentration of N in Precipitation.
C_N_Ql_in = 2.0e6   # ug/m3 Concentration of N in lateral inflow.
C_N_Qk_in = 0.      # ug/m3 Concentration of N in vertical inflow.    

### 3.2 NH$_3$ deposition

Next, let's add the dry deposition rate you calculated in the Basic Skills Air Quality. If you worked correctly through your *Basic Skills Air Quality* notebook you should have saved a matrix at it's end. In that matrix you saved the calculated dry deposition flux $F_{NH_3}$ ($\mu$g m$^{-2}$s$^{-1}$). Here, you will first load that matrix and the necessary content, i.e., the dry deposition rates for three stations that the calculations were made for. Then, you will convert the units to g h$^{-1}$ using the surface area of the lake and 3600 h$^{-1}$. Next, you will convert the mass of NH$_3$ to the mass of only the nitrogen N, using the atom masses of N and H. Finally, you will calculate the wet deposition rate by multiplying the precipitation rate $P$ in m$^3$ h$^{-1}$ with concentration in precipitation C_N_P (that was defined in the previous cell box). The precipitation rate, or simply precipitation, we already used in the Integration Skills 1. 

### Select your input files from the output of BasicSkills_AirQuality and IntegrationSkills_1

Here we offer different options to work with default files or with files you produced yourself:

<ul>
<li><p style="background:powderblue;color:black">$F_{NH_3}$ data:<br>
    - work with output from Basic Skills Air Quality (NH3 deposition over a surface determined for your group's land unit).<br>
    - work with default files supplied by us (assuming deposition over a water surface: rb = rc = 0 s/m and z0 = 0.0001 m). This path is given in the cell box below.</li><br>
<li><p style="background:powderblue;color:black">Water balance data:<br>    
    - work with output from IS 1 (where you defined E_mak, Ql_in, Qk_in and Q_k_out for your group's land unit).<br>
    - work with default files supplied by us (assuming flow rates representative for the Forum pond). This path is given in the cell box below.</li></ul>

In [ ]:
# Below you can select whether or not you want to use your own output datafiles.
USE_MY_AMMONIA_DEPOSITION_FILE = False  # Advise: set to False, then it will use a file with F_NH3 over a water surface
USE_MY_WATER_BALANCE_FILE      = True   # Advise: set to True, to use the settings you made for Qlin and Qkin during IS 1.

# Select input files:
if USE_MY_AMMONIA_DEPOSITION_FILE:
    datapath      = '../BS_AirQuality/'
else:    
    datapath      = os.getenv('TEACHER_DIR') + '/JHL_data/IC_data/'
outfilename_BS_AQ = datapath + 'Data_Output_BasicSkills_AirQuality.csv'   # Use your own F_NH3 files, provided by us

if USE_MY_WATER_BALANCE_FILE:
    datapath      = '../IS_1_WaterBalance/'
else:
    datapath      = os.getenv('TEACHER_DIR') + '/JHL_data/IC_data/'
outfilename_IS1   = datapath + 'Data_Output_IntegrationSkills_1.csv'                     # Use your own output files from IS 1.


print('You will use the following files:')
print(outfilename_BS_AQ)
print(outfilename_IS1)

In [ ]:
# Load the dry deposition rate calculated in the Basic Skills Air Quality. 
# Load the dry deposition rate for the three stations i.e., Wekerom, Vredepeel and Zegveld
# as F_NH3_W, F_NH3_V, and F_NH3_Z, respectively. 
df_drydepo = pd.read_csv(outfilename_BS_AQ, sep=';', 
    index_col='date-time', usecols=['date-time', 'F_NH3_Wekerom', 'F_NH3_Vredepeel', 'F_NH3_Zegveld'], parse_dates=['date-time'])

# Load the matrix you saved at the end of Integration Skills - part 1. 
# Load only the necessary data. Here we load the precipitation P, volume V, 
# evaporation E, lateral inflow and outflow, vertical inflow and outflow. Then, we load 
# volume, evaporation and lateral outflow calculated using Makking method (_mak).
# Next, we load volume, in- and outflows using the real calculation (when we used more 
# realistic inflows), and sensitivity case.
df_meteo = pd.read_csv(outfilename_IS1, sep=';', 
    index_col='date-time', usecols=['date-time', 'P', 
       'V'     , 'E'     ,'Ql_in'     ,'Ql_out'     , 'Qk_in'      , 'Qk_out',
       'V_mak' , 'E_mak'              ,'Ql_out_mak' ,
       'V_real',          'Ql_in_real','Ql_out_real', 'Qk_in_real', 'Qk_out_real',
       'V_sens',          'Ql_in_sens','Ql_out_sens', 'Qk_in_sens', 'Qk_out_sens' ],
                        parse_dates=['date-time'])


# Calculate the wet deposition rate. Note that the precipitation is already converted
# from mm h$^{-1}$ to m$^3$ h$^{-1}$ using the pond's surface area. 
# We did this in Integration Skills - part 1.
df_meteo['dNdt_FNH3_wet'] = df_meteo['P'] * C_N_P

# Convert the units of the deposition rate to ugNH3/m2/h to gN/h. Therefore, set the area of the lake 
# (A; m^2), convert the seconds to hours, use the ratio 17/14 to convert the dry deposition 
# of NH3 to dry deposition of nitrogen N. 
# Lastly, let's not forget to multiply with 10^-6 to convert ug to g.
df_drydepo['dNdt_FNH3_dry']   = df_drydepo['F_NH3_Wekerom'  ]*A*3600.*17./14  # Use Wekerom
#df_drydepo['dNdt_FNH3_dry']  = df_drydepo['F_NH3_Vredepeel']*A*3600.*17./14  # Use Vredepeel
#df_drydepo['dNdt_FNH3_dry']  = df_drydepo['F_NH3_Zegveld'  ]*A*3600.*17./14. # Use Zegveld

### 3.3 Denitrification

Denitrification will not be quantified in the lab in the Integration Course. Nonetheless, you can add a term to make a possibility to do this quantification during PGO part of this course. You can consult Edwin Peeters (Acquatic Ecology) about the meaning of denitrification and it's influence on the water quality. 

<ul>
<li><p style="background:powderblue;color:black">If you’d want to add a denitrification term, denitrification in water is about 11 $\mu$mol N m$^{-2}$ h$^{-1}$ at $T=20$ $^\circ$C. This number may help you decide if denitrification could be an important term.</li></ul> 

In [ ]:
# Add denitrification as a constant term (dNdt_den; ugN/h).
df_meteo['dNdt_den'] = 11.*14.*A*0.000001

This fixed value is of course a very simplistic approach: in reality the denitrification rate depends on temperature and nitrogen content. We will use this value for now. You can make it interactive in the PBL part of the course.

### 3.4 Inflows and outflows

Now, let's implement the inflows of N via the lateral $Q_{l,in}$ and vertical $Q_{k,in}$ inflows. That you can do by multiplying the vertical and lateral flow rates (that you loaded together with precipitation in the box above) with the concentrations in the inflow C_N_l_in and C_N_k_in. 

### Exercise 1: Change of nitrogen in time

The outflow of nitrogen N depends on the concentration inside the pond, which is not available yet. At this point, you have implemented most of the model terms, albeit with unrealistic values for some tendency terms. We will first finish the model formulation, before making those terms more realistic.

In this exercise, after we performed time integration to calculate the nitrogen mass in the Forum Ponds, we will look at the change of nitrogen concentration in the ponds. The concentration is calculated as a ratio of nitrogen mass and water volume in the ponds. Therefore, let's make the time integration, plot and analyze our result.

### Question 1.1: 

Does the N concentration in the pond vary around an equilibrium value or do you see a consistent increase or decrease in N concentration (apart from the seasonal variability)?<br>

*To answer question 1.1 do the following*

<ul>
<li><p style="background:powderblue;color:black">Set up the initial N concentration and mass in the pond `C_N_pond_ini` and `mN_ini`.</li><br>
<li><p style="background:powderblue;color:black">Initialize the arrays for mass of N and lateral, vertical and total N outflow.</li><br>
<li><p style="background:powderblue;color:black">Calculate lateral and vertical inflows of N into the pond.</li><br>
<li><p style="background:powderblue;color:black">Finally, integrate the model in time by computing the mass of the next time step using the equation introduced as: $m_{t+\Delta t} = m_t + dm/dt \times \Delta t$ </li><br>
<li><p style="background:powderblue;color:black">In the second box below, plot the result for the nitrogen concentration rate.</li><ul>

- Write your results in the third box below. To answer the question, fill in the following sentences.<br>

`x: The N concentration (x = 'has/does not have') a seasonal cycle.`<br>
`y: The N concentration has a (y = 'maximum/minimum') in the summer.`<br>

In [ ]:
# Initial conditions
C_N_pond_ini                            = 2e6    #  Set initial N concentration in the pond (C_N_pond_ini; g/m^3).
mN_ini                                  = C_N_pond_ini * V_ini

# Calculate lateral and vertical  inflows of N.
df_meteo['dNdt_Ql_in']                  = df_meteo['Ql_in'] * C_N_Ql_in # ug/h lateral  inflow
df_meteo['dNdt_Qk_in']                  = df_meteo['Qk_in'] * C_N_Qk_in # ug/h vertical inflow


# Initialise arrays containing variables. Set the tendency terms to 0 ugN/h at the first timestep.
it                                      = df_meteo.index[0]
df_meteo.at[it, 'mN'         ]          = mN_ini   # ug/m3  Initial N mass in the pond
df_meteo.at[it, 'dNdt_Ql_out']          = 0.       # ug N/h Tendency term due to lateral  outflow 
df_meteo.at[it, 'dNdt_Qk_out']          = 0.       # ug N/h Tendency term due to vertical outflow
df_meteo.at[it, 'dNdt_tot'   ]          = 0.       # ug N/h Tendency term (total)

# Time integration.
nt                                      = len(df_meteo)
for it in range(1,nt):
    it_now                              = df_meteo.index[it  ]
    it_prev                             = df_meteo.index[it-1]
    C_N_pond                            = df_meteo.at[it_prev, 'mN']/df_meteo.at[it_prev,'V']
    df_meteo.at[it_now, 'dNdt_Ql_out']  = df_meteo.at[ it_now, 'Ql_out'] * C_N_pond
    df_meteo.at[it_now, 'dNdt_Qk_out']  = df_meteo.at[ it_now, 'Qk_out'] * C_N_pond
    df_meteo.at[it_now, 'dNdt_tot'   ]  = df_drydepo.at[it_now, 'dNdt_FNH3_dry'] + df_meteo.at[it_now, 'dNdt_FNH3_wet'] \
                                        + df_meteo.at[ it_now, 'dNdt_Ql_in'   ] - df_meteo.at[it_now, 'dNdt_Ql_out'  ] \
                                        + df_meteo.at[ it_now, 'dNdt_Qk_in'   ] - df_meteo.at[it_now, 'dNdt_Qk_out'  ] \
                                        - df_meteo.at[ it_now, 'dNdt_den'     ]
    df_meteo.at[it_now, 'mN'         ]  = df_meteo.at[ it_prev,'mN']            + df_meteo.at[it_now, 'dNdt_tot']*dt

In [ ]:
# Plot the time series nitrogen concentration. 
# Plot Time [year] on x-axis and Concentration [ug m^-3] on y-axis. 
C_N_pond = df_meteo['mN']/df_meteo['V']
fig1 = (C_N_pond).iplot(asFigure=True, xTitle='Time [year]', yTitle='Nitrogen concentration [ug/m3]', width=2) 
fig1.show()

In [ ]:
# Write your results:
# x: The N concentration (x = 'has/does not have') a seasonal cycle.
# y: The N concentration has a (y = 'maximum/minimum') in the summer.
x  = 
y  = 

if nemo.check_my_answer_IS201(x, y):
    nemo.app.run()

nemo.app.canvas

### Exercise 2: Finding the equilibrium N concentration

If the concentration shows a gradual increase (decrease) in time, the initial concentration was set too low (high). Ideally, you would want the model to find this equilibrium concentration itself, but the 8 years time period you apply the model to is probably too short.

You may help the model find its equilibrium concentration by setting the initial concentration inside the pond (C_N_pond_ini) to a larger (smaller) value, e.g., by using the concentration at the end of the time series as a new initial concentration. You may need to iterate several times. In this way, you set the initial concentration to a value close to the equilibrium concentration. Therefore, go to the cell box under section 3.4 and change your initial N concentration until the result on the figure gives approximately the equilibrium state.

### Question 2.1: 

When changing the initial concentration of nitrogen, did your concentration reach the equilibrium state? <br>
What initial value did you chose to prove that? <br>

*To answer the question 2.1 fill in the following sentences:*

- Write your answer in the cell box below.<br>
`The N concentration reaches the quasi-equilibrium state when we (x = 'decrease/increase') the initial N concentration.`<br>
`The new initial N concentration is (y = number) ug/m3.`

In [ ]:
# Write your answers.
# x: The N concentration reaches the quasi-equilibrium state when we 
# (x = 'decrease/increase') the initial N concentration.
# y: The new initial N concentration is about (y = number).
x = 
y = 

if nemo.check_my_answer_IS202(x,y):
    nemo.app.run()

nemo.app.canvas

### Exercise 3: Water balance: Simplistic, Makkink Evaporation or Realistic.

In Integration Skills 1, you performed several water balance model runs.

1. First, you made rather simplistic calculations with only precipitation. Evaporation and in- and outflow were assumed to be constant or zero. Above, you used the water balance terms from this run ($V$). 
1. Second, you calculated the evaporation (and later the water volume in the ponds) using Makkink's method to make a water volume more realistic ($V_{mak}$). 
1. Third, you implemented realistic values for lateral and vertical in- and outflows ($V_{real}$).
1. Fourth, you performed a sensitivity study to estimate the impact of several settings/parameters ($V_{sens}$).

In this exercise, you are asked to use the volume of one of the runs 2-4 to calculate the nitrogen mass in the forum ponds. You can do the experiment as follows:

- Edit the second cell below.
- replace the variables related to Volume V_???, E_??? and mN_??? with the ones associated with that run(e.g. V_mak, V_real,...) (see table in the next cell).
- perform the calculations and plot the results.

Note that most of the job is already done for you in the cell under Exercise 1: Change of nitrogen in time. Also, make sure that the initial N concentrations (C_N_pond_ini) in both cells are the same. 

In [ ]:
# Initial conditions
C_N_pond_ini                            = 3.5e6    #  Set initial N concentration in the pond (C_N_pond_ini; g/m^3).
mN_ini                                  = C_N_pond_ini * V_ini

# Calculate lateral and vertical  inflows of N.
df_meteo['dNdt_Ql_in']                  = df_meteo['Ql_in'] * C_N_Ql_in # ug/h lateral  inflow
df_meteo['dNdt_Qk_in']                  = df_meteo['Qk_in'] * C_N_Qk_in # ug/h vertical inflow

# Initialise arrays containing variables. Set the tendency terms to 0 ugN/h at the first timestep.
it                                      = df_meteo.index[0]
df_meteo.at[it, 'mN_???'     ]          = mN_ini   # ug/m3  Initial N mass in the pond
df_meteo.at[it, 'dNdt_Ql_out']          = 0.       # ug N/h Tendency term due to lateral  outflow 
df_meteo.at[it, 'dNdt_Qk_out']          = 0.       # ug N/h Tendency term due to vertical outflow
df_meteo.at[it, 'dNdt_tot'   ]          = 0.       # ug N/h Tendency term (total)

# Time integration.
nt                                      = len(df_meteo)
for it in range(1,nt):
    it_now                              = df_meteo.index[it  ]
    it_prev                             = df_meteo.index[it-1]
    C_N_pond                            = df_meteo.at[ it_prev, 'mN_???']/df_meteo.at[it_prev,'V_???']
    df_meteo.at[it_now, 'dNdt_Ql_out']  = df_meteo.at[  it_now, 'Ql_out_???'] * C_N_pond
    df_meteo.at[it_now, 'dNdt_Qk_out']  = df_meteo.at[  it_now, 'Qk_out'] * C_N_pond
    df_meteo.at[it_now, 'dNdt_tot'   ]  = df_drydepo.at[it_now, 'dNdt_FNH3_dry'] + df_meteo.at[it_now, 'dNdt_FNH3_wet'] \
                                        + df_meteo.at[  it_now, 'dNdt_Ql_in'   ] - df_meteo.at[it_now, 'dNdt_Ql_out'  ] \
                                        + df_meteo.at[  it_now, 'dNdt_Qk_in'   ] - df_meteo.at[it_now, 'dNdt_Qk_out'  ] \
                                        - df_meteo.at[  it_now, 'dNdt_den'     ]
    df_meteo.at[it_now, 'mN_???'     ]  = df_meteo.at[  it_prev,'mN_???']        + df_meteo.at[it_now, 'dNdt_tot']*dt

In [ ]:
# Plot the time series of nitrogen concentration. 
# Plot Time [year] on x-axis and Concentration [ug m^-3] on y-axis. 
C_N_pond       = df_meteo['mN'    ]/df_meteo['V'    ]
C_N_pond_real  = df_meteo['mN_???']/df_meteo['V_???']
df_C_N         = pd.concat([C_N_pond, C_N_pond_real],axis=1, sort=False)
df_C_N.columns = ['Simplistic', '???']

fig2 = (df_C_N).iplot(asFigure=True, xTitle='Time [year]', yTitle='Concentration [ug/m3]', width=2) 
fig2.show()

### Question 3.1: 

Which model run did you use?<br>
How does your calculation differ compared to the original 'simplistic' calculation (after adjusting the initial concentration)? <br>

*To answer question 3.1 fill in the following sentences:*

- Write your answers in the cell box below. <br>
`x: The realistic simulation shows (y = 'more/less') pronounced seasonal cycle compared to the simplistic simulation.`<br>
`y: The maximum N concentrations seen in (y = 'Season', e.g. 'Winter','Spring','Summer','Fall') are similar in both runs, `<br>
`z: but the minima seen in (z = 'Season') are much lower in the realistic runs.`<br>

In [ ]:
# Write your answers.
# x: The realistic simulation shows (y = 'more/less') pronounced seasonal cycle compared to the simplistic simulation.
# y: The maximum N concentrations seen in (y = 'Season', e.g. 'Winter','Spring','Summer','Fall') are similar in both runs, 
# z: but the minima seen in (z = 'Season') are much lower in the realistic runs.
x  = 
y  = 
z  = 

if nemo.check_my_answer_IS203(x,y,z):
    nemo.app.run()

nemo.app.canvas

## 4 Is it safe to swim in the campus ponds?

“Is it safe to swim in the campus ponds?” This is the question we started the series of Integration Skills with. Particularly when the sun shines and air temperature approaches tropical values, algae and bacteria may be a problem for water quality. The next question is how to define ‘safe swimming conditions in (natural) surface water’? Two issues are of main concern here: clear vision under water and health. The standards for safe swimming water are based on these two criteria.

For security reasons, a minimum visibility of 1 m has been defined as the limit. If visibility is lower, it becomes difficult to detect objects (like trees or bicycles) that have been 'fallen' in the water. Also, with low visibility it becomes difficult to assess how deep the water is and that is of importance if one wants to dive. For health reasons, it would be beneficial if not too many sick making bacteria are in the water system. Especially bacteria present in the faeces of humans and animals my turn people very sick. Those bacteria should be limited to a below a maximum level. It is also very annoying if one gets swimmers itch, but this topic is beyond the scope of the course.

### 4.1 Algae and Redfield ratio

In Basic Skills Water Quality 1, you learned what type of algae, blue or green, will become dominant at a certain Redfield ratio. Please refer to the information provided there. Above you have simulated how the N concentration in the pond varies over time. You may use the Phosphorus concentrations indicated in the Introductory slides on Brightspace.

### 4.2 Underwater-light climate

A so-called secchi disk can be used to determine how far light penetrates into the water column. The secchi disk is a black and white coloured disk on a rope with depth marks. To use it, you should slowly sink the disk in the water column, just until you don’t see the disk anymore. Note the depth. Now, you should slowly lift the disk until you see it again. Note the depth again. The average of the two values is the Secchi depth at that moment. Standards for visibility have been set to minimally 0.4 m for all surface waters and 1.0 m for natural swimming waters.

Visibility of the water is depending on different factors. Algae have the capacity to make the water really turbid, but only if enough nutrients are available. Especially phosphorus and nitrogen are important for the growth of these algae. If the concentrations of available phosphorus and nitrogen in the water are known, it is possible to calculate how much algal biomass this will yield, assuming that all available nutrients are taken up by the algae. The algal composition is C106H175O42N16P. Corresponding molar weights are C (12), H (1), O (16), N (14) and P (31) which are all expressed in g/mol. Hoyer et al. (2002) determined the following relationship between Secchi depth and $Chlorophyll-a$ (the chlorophyl absorbing photosynthetically active radiation) content in the water:

\begin{equation*}
10\log(Secchi)= 0.65 – 0.43 × 10\log( Chlorophyll-a )
\end{equation*}

where $Chlorophyll-a$ is expressed in $\mu$g L$^{-1}$ and Secchi depth in m. It is well known that approximately 1$\%$ of the algal biomass is made of $Chlorophyll-a$. If Secchi disk measurements are available, one can easily determine how much $Chlorophyll-a$ there is in the water.

The measured nutrients in the water can be converted into algal biomass (see Basic skills Water Quality for how to calculate) and one knows the amount of $Chlorophyll-a$ that can be made from the available nutrients. Summing up both $Chlorophyll-a$ values and filling in in equation above will give the potential minimum value for visibility. By comparing this value with the standard, one is able to evaluate whether swimming in the pond is safe or not.

### Exercise 4: Chlorophyl and water visibility

In this exercise, we are going to calculate the amount of chlorophyl and water visibility in the Forum Ponds.

### Question 4.1: 

How does a water visibility change compared to the chlorophyl amount in the Forum ponds?<br>
Can you explain the peaks in the chlorophyle seen in summer 2018-2020? <br>

*To answer question 4.1 do the following:*

<ul>
<li><p style="background:powderblue;color:black">In the first cell box below, load the air temperature `df_chlorofyl` in degress and the global radiation `df_rg` in W/m2.</li><br>
<li><p style="background:powderblue;color:black">In the second cell box below, setup some initial values and parameters needed for the model.</li><br>
<li><p style="background:powderblue;color:black">Setup the temperature, nitrogen, phosphorus and nutrient limitations.</li><br>
<li><p style="background:powderblue;color:black">Finally, perform the time integration.</li><br>
<li><p style="background:powderblue;color:black">In the third box below, plot the result for the chlorophyl and secchi rate.</li><ul>

- Write your results in the fourth cell box below. To do so fill in the follwing sentences and select one of the correct answers.<br>
`The visibility is (x = 'proportional/inversely proportional') to the chlorophyl amount.`<br>
`The peak seen the chlorophyl in summers of 2018-2020 happens because of y = :`<br>
`    'a' increased water temperature`<br>
`    'b' increased light level`<br>
`    'c' increased nutrient loads`<br>
`    'd' low water levels` <br>

In [ ]:
# Load the air temperature (Air_temp; degrees).
teacher_dir = os.getenv('TEACHER_DIR')
df_chlorophyl = pd.read_csv(teacher_dir + '/JHL_data/IC_data/Data_Meteo_Veenkampen_2013-2020.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'Ta'], parse_dates=['date-time'])

# Load the global radiation (Rg; W/m^2).
df_rg = pd.read_csv(teacher_dir + '/JHL_data/IC_data/Data_Meteo_Veenkampen_2013-2020.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'Rg'], parse_dates=['date-time'])

In [ ]:
# Set up the initial chlorofyl value.
Chlorophyl_ini    = 1e-4            # ug/l
df_chlorophyl['Chlorophyl'] = Chlorophyl_ini

# Parameter settings.
r                 = 0.5             # Growth rate
l                 = 0.005           # Loss rate
Tmin              = 0.0             # oC Min temperature for algae growth
Topt              = 20.0            # oC Optimal temperature
Tmax              = 40.0            # oC Max temperature for algae growth
hN                = 0.40            # Sensitivity of algae growth to N concentration
hP                = 0.15            # Sensitivity of algae growth to P concentration
ha                = 15.             # Crowding factor
depth             = 1.5             # Depth in m

# Temperature limitation.
Ta                = df_chlorophyl['Ta'] # =IF(H9>0;IF(H9<Topt;H9*1/Topt;1-(H9-Topt)*1/Topt);0)
Tlim              = Ta * 0.0        # init
Tlim[(Ta >  Tmin) & (Ta < Topt)] = Ta/Topt
Tlim[(Ta >= Topt) & (Ta < Tmax)] = 1-(Ta-Topt)/Topt

# Nitrogen limitation.
Llim              = df_rg.copy()
Llim[df_rg['Rg'] >  800] = 1
Llim[df_rg['Rg'] <= 800] = df_rg/800.
C_N               = C_N_pond_real  #* from_ugperm3_to_mgperl
Nlim              = C_N  / (hN + C_N)

# Phosphorus limitation.
C_P               = 5   # ug/l (fixed value for now, adapt if you have better measurements)
Plim              = C_P / (hP + C_P)

# Nutrient limitation.
Nutlim            = np.minimum(Nlim,Plim)

# Time integration.
nt = len(df_chlorophyl)
for it in range(1,nt):
    it_now  = df_chlorophyl.index[it  ]
    it_prev = df_chlorophyl.index[it-1]
      
    df_chlorophyl.at[it_now, 'Chlorophyl'] = df_chlorophyl.at[it_prev, 'Chlorophyl'] \
        + (r * df_chlorophyl.at[it_prev,'Chlorophyl'] * Llim.at[it_now, 'Rg'] * Tlim[it_now] * \
                                                         Nutlim[it_now] * (ha/(ha+df_chlorophyl.at[it_prev, 'Chlorophyl']))) \
                                                      -  l * df_chlorophyl.at[it_prev,'Chlorophyl']
    
df_chlorophyl['Secchi'] = np.minimum(10**(0.65-0.43*np.log10(df_chlorophyl['Chlorophyl'])), depth)

In [ ]:
fig4 = df_chlorophyl[['Chlorophyl', 'Secchi']].iplot(asFigure=True, subplots=True, shape=(2,1), shared_xaxes=True,
    layout=dict(yaxis=dict(title='Chlorophyl [ug/l]'), xaxis=dict(title=' ')))
fig4['layout']['yaxis2'].update({'title':'Secchi [m]'})
fig4['layout']['xaxis2'].update({'title':'Time [years]'})
fig4.show()

In [ ]:
# Write your results.
# x: The visibility is (x = 'proportional/inversely proportional') to the chlorophyl amount.
# y: The peak seen the chlorophyl in summers of 2018-2020 happens --primarily-- because of: 
# y = :
#      'a' increased water temperature
#      'b' increased light level
#      'c' increased nutrient loads
#      'd' low water levels
x = 
y = 

if nemo.check_my_answer_IS204(x,y):
    nemo.app.run()

nemo.app.canvas

Dear students, we have come to the end of this part of Integration skills. We ask you to follow the usual procedure when logging out of the notebook. 

Thus, to limit the data storage please do the following:
- go to *KERNEL* --> *RESTART and CLEAR OUTPUT*. This removes the interactive graphs from your notebook, but leaves your answers intact.
- go to *FILE* --> *SAVE and CHECKPOINT*. This saves your answers in the notebook
- go to *FILE* --> *CLOSE and HALT*, to shutdown the notebook. You can always come back later and restart.

We hope you enjoyed the exercises and that the laened material will help you in the group part of this course. 

### Save your data for later

In [ ]:
df_meteo.to_csv('Data_Output_IntegrationSkills_2.csv', sep=';')
df_chlorophyl.to_csv('Data_Output_Algae.csv', sep=';')

In [ ]:
#Check out the columns in these data files:
print(df_meteo.keys())
print(df_chlorophyl.keys())